In [1]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1512\2936652086.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


conclusion: 

Feature Selection: CCI, ADX, Chaikin A/D
Model Selection: best using SVC with poly kernel
Hyperparameter Tuning: d = 2 c = 0.3

Report

1. SVM -> stock price -> goblok tebak all up or down
2. Data collection -> jelasin features dari alphavantage + feature buatan sendiri (short SMA / long SMA ratio)
3. Model Selection -> analyze correlation  bilkang not linearly separable ...
4. Feature Selection
5. Hyperparameter tuning

stevan: 
- feature buatan sendiri (short SMA / long SMA ratio)
- training data selection
- improve acc to >70%

In [2]:
'''
CONST
'''
FILEPATH_STOCK_PRICE = "./tr_eikon_eod_data.csv"
STOCK_NAMES = ['AAPL.O', 'MSFT.O', 'MSFT.O', 'AMZN.O', 'MSFT.O']

In [3]:
''' 
UTILS
'''
def sequential_train_test_split(data, ratio = 0.8):
    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df[split_index:]

def calculate_return(prediction, data) :
    data = data
    prediction = pd.DataFrame(prediction)
    temp = pd.DataFrame(data)
    temp['log_return'] = np.log(temp / temp.shift(1))
    temp['returne'] = prediction.shift(1) * temp['log_return'].values.reshape(-1, 1)
    temp.dropna(inplace = True) 

    return np.exp(temp[['log_return', 'returne']].sum())


In [4]:
'''
BENCHMARK
'''

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    return (prediction == test).sum() / len(prediction)


In [5]:
''' 
MODEL
'''

def predict_position(train, test):
    # Train Hidden Markov Model
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(train.reshape(-1, 1))

    # Predict hidden states for test data
    hidden_states = model.predict(test.reshape(-1, 1))

    # Define a function to map hidden states to rise or fall
    def map_to_rise_or_fall(state):
        return 1 if state == 0 else -1

    # Map hidden states to rise or fall
    predicted_rise_fall = np.array(list(map(map_to_rise_or_fall, hidden_states)))

    return predicted_rise_fall

In [7]:
apple_data = pd.read_csv('microsoft_output.csv', index_col = 0, parse_dates = True).dropna()
print(apple_data)
apple_data['SMA_EMA_ratio'] = apple_data['SMA'] / apple_data['EMA']
SMA_short = 7
SMA_long = 20
apple_data['SMA_Short'] = apple_data['MSFT.O'].rolling(SMA_short).mean()
apple_data['SMA_Long'] = apple_data['MSFT.O'].rolling(SMA_long).mean()
apple_data['SMA_Long_Short'] = apple_data['SMA_Short'] / apple_data['SMA_Long']
apple_data['data'] = apple_data['MSFT.O']
apple_data['direction'] = np.where(apple_data['MSFT.O'] <= apple_data['MSFT.O'].shift(-1), 1, -1)

apple_data

            MSFT.O    SlowK    SlowD      RSI      ADX       CCI  Aroon Down  \
Date                                                                           
2010-04-01  30.950  22.6012  20.0644  47.6624  36.1135 -200.4061    100.0000   
2010-06-01  30.770  58.4734  43.3955  22.6915  60.6866  -57.2855     78.5714   
2010-07-01  30.452   9.2413   7.7560  20.6368  37.6466 -110.9580    100.0000   
2010-11-01  30.270  84.9747  88.4270  73.8351  41.7313  143.9476      0.0000   
2010-12-01  30.070  59.9617  43.7910  46.6624  30.6007  117.2058     85.7143   
...            ...      ...      ...      ...      ...       ...         ...   
2018-06-25  98.390  36.1002  51.6045  47.7111  18.6535 -211.9001    100.0000   
2018-06-26  99.080  28.9721  40.1854  41.0082  20.4706 -112.7838     92.8571   
2018-06-27  97.540  19.8773  28.3165  46.3974  23.2159 -141.2104     85.7143   
2018-06-28  98.630  25.9274  24.9256  35.8605  25.7901 -102.5560    100.0000   
2018-06-29  98.610  30.3503  25.3850  45

,MSFT.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,30.950,22.6012,20.0644,47.6624,36.1135,-200.4061,100.0000,64.2857,-1.700326e+08,-7.207873e+09,22.5763,22.4974,1.003507,NaN,NaN,NaN,30.950,-1
2010-06-01,30.770,58.4734,43.3955,22.6915,60.6866,-57.2855,78.5714,14.2857,-7.288506e+08,-7.440150e+09,20.4367,20.3482,1.004349,NaN,NaN,NaN,30.770,-1
2010-07-01,30.452,9.2413,7.7560,20.6368,37.6466,-110.9580,100.0000,42.8571,-1.307756e+09,-7.786347e+09,19.1478,18.8035,1.018310,NaN,NaN,NaN,30.452,-1
2010-11-01,30.270,84.9747,88.4270,73.8351,41.7313,143.9476,0.0000,100.0000,-7.992383e+08,-7.177443e+09,19.7969,19.9471,0.992470,NaN,NaN,NaN,30.270,-1
2010-12-01,30.070,59.9617,43.7910,46.6624,30.6007,117.2058,85.7143,0.0000,-1.158711e+09,-7.252644e+09,19.6940,19.7209,0.998636,NaN,NaN,NaN,30.070,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,98.390,36.1002,51.6045,47.7111,18.6535,-211.9001,100.0000,14.2857,-7.445996e+08,-3.823874e+09,94.8803,94.6967,1.001939,100.522857,100.3715,1.001508,98.390,1
2018-06-26,99.080,28.9721,40.1854,41.0082,20.4706,-112.7838,92.8571,7.1429,-7.177024e+08,-3.837423e+09,94.6660,94.3661,1.003178,100.372857,100.4100,0.999630,99.080,-1
2018-06-27,97.540,19.8773,28.3165,46.3974,23.2159,-141.2104,85.7143,0.0000,-7.490008e+08,-3.865377e+09,94.4649,94.2254,1.002542,99.898571,100.3690,0.995313,97.540,1


In [8]:
scaler = MinMaxScaler()

# Normalize the selected columns
normalized_columns = scaler.fit_transform(apple_data.iloc[:, :-2])

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_columns, columns=apple_data.columns[:-2])

# Display the normalized DataFrame
apple_data.iloc[:, :-2] = normalized_df
apple_data


,MSFT.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,0.099899,0.220267,0.172315,0.434657,0.527014,0.148550,1.000000,0.642857,0.860606,0.147638,0.047850,0.045590,0.565876,NaN,NaN,NaN,30.950,-1
2010-06-01,0.097635,0.588096,0.419744,0.109860,1.000000,0.380652,0.785714,0.142857,0.684550,0.092388,0.024021,0.021571,0.585388,NaN,NaN,NaN,30.770,-1
2010-07-01,0.093634,0.083276,0.041783,0.083134,0.556523,0.293610,1.000000,0.428571,0.502164,0.010040,0.009666,0.004308,0.908847,NaN,NaN,NaN,30.452,-1
2010-11-01,0.091344,0.859837,0.897308,0.775086,0.635146,0.706997,0.000000,1.000000,0.662374,0.154876,0.016895,0.017089,0.310166,NaN,NaN,NaN,30.270,-1
2010-12-01,0.088827,0.603357,0.423938,0.421650,0.420903,0.663629,0.857143,0.000000,0.549121,0.136989,0.015749,0.014561,0.453020,NaN,NaN,NaN,30.070,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.948415,0.358684,0.506801,0.435291,0.190941,0.129910,1.000000,0.142857,0.679588,0.952565,0.853120,0.852466,0.529542,0.988560,0.999495,0.464984,98.390,1
2018-06-26,0.957096,0.285593,0.385700,0.348106,0.225917,0.290649,0.928571,0.071429,0.688062,0.949342,0.850734,0.848771,0.558253,0.986629,1.000000,0.454168,99.080,-1
2018-06-27,0.937720,0.192336,0.259829,0.418203,0.278759,0.244549,0.857143,0.000000,0.678201,0.942693,0.848494,0.847198,0.543511,0.980522,0.999462,0.429304,97.540,1


In [9]:
# apple_data = apple_data[['SlowK', 'RSI', 'CCI', 'Aroon Down', 'OBV', 'direction']]

In [10]:
train_data, test_data = sequential_train_test_split(apple_data)
print(test_data)
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]

[[ 0.47798188  0.7577973   0.69911723 ...  0.50190022 61.
  -1.        ]
 [ 0.47785606  0.91508971  0.77938622 ...  0.53671023 60.99
  -1.        ]
 [ 0.47332662  0.93264024  0.86966537 ...  0.56891324 60.63
  -1.        ]
 ...
 [ 0.93772018  0.19233605  0.25982934 ...  0.42930404 97.54
   1.        ]
 [ 0.95143432  0.25437302  0.22386849 ...  0.40925916 98.63
  -1.        ]
 [ 0.95118269  0.29972489  0.22874047 ...  0.38350627 98.61
  -1.        ]]


In [11]:
X_train

array([[5.92601912e-02, 6.82671258e-01, 6.85461036e-01, ...,
        7.35542151e-02, 2.55511779e-01, 2.77200000e+01],
       [6.29089079e-02, 5.74181151e-01, 3.58221396e-01, ...,
        7.16255178e-02, 2.41724535e-01, 2.80100000e+01],
       [6.42929039e-02, 8.80033756e-01, 8.75718761e-01, ...,
        6.98870662e-02, 2.23410729e-01, 2.81200000e+01],
       ...,
       [4.34323100e-01, 7.06747358e-01, 5.89320446e-01, ...,
        4.36617039e-01, 4.51810337e-01, 5.75300000e+01],
       [4.30800201e-01, 6.82258026e-01, 6.68836471e-01, ...,
        4.36826965e-01, 4.38901639e-01, 5.72500000e+01],
       [4.61122295e-01, 6.70774711e-01, 6.81518057e-01, ...,
        4.38696621e-01, 4.59967817e-01, 5.96600000e+01]])

In [12]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', gamma=0.7, C=1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
benchmark_accuracy(pred, y_test)

0    0.55615
dtype: float64

In [13]:
best_acc = 0
best_g = -1
best_c = -1
best_return = 0
bestgg = -1
bestcc = -1
log_return = 0  

selected_features = [4, 5, 9]
dat = X_test[:,-1]

for g in range(1, 11, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='rbf', gamma=g / 10, C=c / 10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)
        log_return = temp.values[0]
        if acc[0] > best_acc:
            best_acc = acc[0]
            best_g = g
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestgg = g
            bestcc = c

print(f"best acc: {best_acc}, g: {best_g}, c: {best_c}")
print(f"best ret: {best_return}, g: {bestgg}, c: {bestcc}")

best acc: 0.56951871657754, g: 8, c: 41
best ret: 1.7675104628259348, g: 8, c: 43


In [14]:
print(log_return)

1.6165573770491792


In [15]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]
dat = X_test[:,-1]

for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='sigmoid', C= c/10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5614973262032086, c: 1
best ret: 1.6165573770491792, c: 1


In [16]:
best_acc = 0
best_d = -1
best_c = -1
best_return = 0
bestdd = -1
bestcc = -1

selected_features = [1, 5, 11, 13]
dat = X_test[:,-1]

for d in range(1, 5, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree=d, C=c/10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_d = d
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestdd = d
            bestcc = c
    
print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")
print(f"best ret: {best_return}, d : {bestdd}, c: {bestcc}")

best acc: 0.56951871657754, d: 1, c: 35
best ret: 1.8442344384533584, d : 1, c: 35


In [17]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]

# for d in range(1, 5, 1):
for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='poly', degree=2, C = c / 10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5614973262032086, c: 1
best ret: 1.6165573770491792, c: 1


In [18]:
# Feature Engineering

# Feature Selection

In [19]:
X = apple_data.dropna().iloc[:, :-1]
y = apple_data.dropna().iloc[:, -1]

In [20]:
X

,MSFT.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data
Date,,,,,,,,,,,,,,,,,
2010-08-02,0.059260,0.682671,0.685461,0.587958,0.165948,0.618039,0.357143,0.857143,0.648204,0.105585,0.015725,0.014135,0.495216,0.064631,0.073554,0.255512,27.72
2010-09-02,0.062909,0.574181,0.358221,0.317266,0.454964,0.433229,0.857143,0.142857,0.559094,0.105693,0.001439,0.000000,0.473427,0.061891,0.071626,0.241725,28.01
2010-11-02,0.064293,0.880034,0.875719,0.795918,0.691745,0.688833,0.000000,1.000000,0.679513,0.165875,0.018427,0.018786,0.294885,0.059040,0.069887,0.223411,28.12
2010-12-02,0.061902,0.769798,0.579200,0.606181,0.429127,0.806426,0.785714,1.000000,0.578030,0.153778,0.016042,0.015692,0.365636,0.056778,0.068233,0.212690,27.93
2010-02-16,0.067187,0.697589,0.560187,0.328880,0.510031,0.629395,0.571429,0.214286,0.843222,0.133273,0.033355,0.033357,0.346063,0.057090,0.066973,0.235538,28.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.948415,0.358684,0.506801,0.435291,0.190941,0.129910,1.000000,0.142857,0.679588,0.952565,0.853120,0.852466,0.529542,0.988560,0.999495,0.464984,98.39
2018-06-26,0.957096,0.285593,0.385700,0.348106,0.225917,0.290649,0.928571,0.071429,0.688062,0.949342,0.850734,0.848771,0.558253,0.986629,1.000000,0.454168,99.08
2018-06-27,0.937720,0.192336,0.259829,0.418203,0.278759,0.244549,0.857143,0.000000,0.678201,0.942693,0.848494,0.847198,0.543511,0.980522,0.999462,0.429304,97.54


In [21]:
y

Date
2010-08-02    1
2010-09-02    1
2010-11-02   -1
2010-12-02    1
2010-02-16    1
             ..
2018-06-25    1
2018-06-26   -1
2018-06-27    1
2018-06-28   -1
2018-06-29   -1
Name: direction, Length: 1868, dtype: int32

In [22]:
best_acc = 0
best_ret = 0

dat = X_test[:,-1]
for i in range(len(apple_data.columns) - 1):
    print(X_train[:,i])
    curr_X_train = X_train[:,i].reshape(-1, 1)
    curr_X_test = X_test[:,i].reshape(-1, 1)
    model = SVC(kernel='rbf', gamma= 1.0, C= 1.0)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp =calculate_return(pred, dat).values[1]
    if temp > best_ret:
        best_ret = temp
    
    print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")
print(best_ret)

[0.05926019 0.06290891 0.0642929  ... 0.4343231  0.4308002  0.46112229]
feature 0: acc: 0.561 up: 1.0
[0.68267126 0.57418115 0.88003376 ... 0.70674736 0.68225803 0.67077471]


feature 1: acc: 0.551 up: 0.668
[0.68546104 0.3582214  0.87571876 ... 0.58932045 0.66883647 0.68151806]
feature 2: acc: 0.556 up: 0.679
[0.58795809 0.31726618 0.79591839 ... 0.48310842 0.49064469 0.83489656]
feature 3: acc: 0.561 up: 1.0
[0.16594775 0.45496419 0.69174505 ... 0.07103907 0.08966356 0.13391886]
feature 4: acc: 0.561 up: 1.0
[0.61803863 0.4332286  0.68883271 ... 0.52503406 0.38325129 0.90832865]
feature 5: acc: 0.561 up: 1.0
[0.357143 0.857143 0.       ... 0.714286 0.642857 0.571429]
feature 6: acc: 0.561 up: 1.0
[0.857143 0.142857 1.       ... 0.5      0.428571 1.      ]
feature 7: acc: 0.537 up: 0.912
[0.64820355 0.55909414 0.67951346 ... 0.43156822 0.41598714 0.44120143]
feature 8: acc: 0.564 up: 0.885
[0.10558471 0.10569306 0.16587523 ... 0.76351118 0.76788835 0.75575617]
feature 9: acc: 0.561 up: 1.0
[0.01572473 0.00143894 0.01842664 ... 0.37581539 0.37557371 0.37802391]
feature 10: acc: 0.561 up: 1.0
[0.01413499 0.         0.0187863  ... 0.37487441 0.37505993 0.38032

In [23]:
best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(len(apple_data.columns) - 1):
    for j in range(len(apple_data.columns) - 1):
        selected_features = [i, j]
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

best acc: 0.5721925133689839 features: [2, 3]
best ret: 1.6969156514960004 features: [8, 12]


In [24]:
print(apple_data.columns)

Index(['MSFT.O', 'SlowK', 'SlowD', 'RSI', 'ADX', 'CCI', 'Aroon Down',
       'Aroon Up', 'OBV', 'Chaikin A/D', 'SMA', 'EMA', 'SMA_EMA_ratio',
       'SMA_Short', 'SMA_Long', 'SMA_Long_Short', 'data', 'direction'],
      dtype='object')


In [25]:
import itertools

all_features = list(range(len(apple_data.columns) - 2))

best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(1, len(apple_data.columns) - 2):
    print(i)
    possible_selected_features = list(itertools.combinations(all_features, i))
    for selected_features in possible_selected_features:
        print(selected_features)
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

1
(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
2
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(2, 10)
(2, 11)
(2, 12)
(2, 13)
(2, 14)
(2, 15)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(3, 10)
(3, 11)
(3, 12)
(3, 13)
(3, 14)
(3, 15)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(4, 9)
(4, 10)
(4, 11)
(4, 12)
(4, 13)
(4, 14)
(4, 15)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(6, 7)
(6, 8)
(6, 9)
(6, 10)
(6, 11)
(6, 12)
(6, 13)
(6, 14)
(6, 15)
(7, 8)
(7, 9)
(7, 10)
(7, 11)
(7, 12)
(7, 13)
(7, 14)
(7, 15)
(8, 9)
(8, 10)
(8, 11)
(8, 12)
(8, 13)
(8, 14)
(8, 15)
(9, 10)
(9, 11)
(9, 12)
(9, 13)
(9, 14)
(9, 15)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(11, 12)
(11, 13)
(11, 14)
(11, 15)
(12, 13)
(12, 14)
(12, 15)
(13, 14

KeyboardInterrupt: 

In [26]:

print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

best acc: 0.6176470588235294 features: (0, 2, 7, 8, 12)
best ret: 2.2721934139614994 features: (2, 3, 5, 7, 8, 15)
